In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import glob 
from PIL import Image
from keras.utils import to_categorical

In [3]:
def load_garbage_dataset(folder, image_size=(32,32), ext=("*.jpg","*.jpeg","*.png"), normalize=True, as_categorical=True):
    """
    Carica immagini da una cartella strutturata per sottocartelle di classe (come CIFAR style).
    Restituisce X (numpy array shape (N,H,W,3)), y (int labels) e class_names.
    Se la cartella non contiene sottocartelle, carica tutte le immagini e assegna etichetta 0.
    considera le cartelle e i file presenti nella cartella Materiale progetto
    """
    folder = os.path.abspath(folder)
    # trova sottocartelle (classi)
    class_names = sorted([d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder,d))])
    filepaths = []
    labels = []

    if class_names:
        for idx, cls in enumerate(class_names):
            cls_folder = os.path.join(folder, cls)
            for pattern in ext:
                for f in glob.glob(os.path.join(cls_folder, pattern)):
                    filepaths.append(f)
                    labels.append(idx)
    else:
        # nessuna sottocartella: prendi tutti i file nella cartella e assegna etichetta 0
        for pattern in ext:
            for f in glob.glob(os.path.join(folder, pattern)):
                filepaths.append(f)
                labels.append(0)
        class_names = ['unknown']
    
    # carica immagini
    images = []
    for f in sorted(filepaths):
        try:
            img = Image.open(f).convert('RGB')
            if image_size is not None:
                img = img.resize(image_size)
            arr = np.asarray(img, dtype=np.float32)
            images.append(arr)
        except Exception as e:
            # salta file corrotti
            continue
    if len(images) == 0:
        return np.empty((0,)+image_size+(3,)), np.array([], dtype=np.int32), class_names
    
    X = np.stack(images, axis=0)
    y = np.array(labels, dtype=np.int32)
    
     # normalizza se richiesto
    if normalize:
        X = X / 255.0

    # one-hot se richiesto
    if as_categorical:
        y_cat = to_categorical(y, num_classes=len(class_names))
        return X, y_cat, class_names
    else:
        return X, y, class_names


# Esempio d'uso (cambia il percorso se necessario)
folder = r"Materiale progetto\Garbage classification"
X, y, class_names = load_garbage_dataset(folder, image_size=(32,32))

print("Classi trovate:", class_names)
print("X shape:", X.shape)
print("y shape:", y.shape)
# mostra prima immagine (se presente)
if X.shape[0] > 0:
    import matplotlib.pyplot as plt
    plt.imshow((X[0] * 255).astype(np.uint8) if X.max() <= 1.0 else X[0].astype(np.uint8))
    plt.title(f"label: {np.argmax(y[0]) if y.ndim>1 else y[0]}")
    plt.axis('off')
    plt.show()

Classi trovate: ['test', 'train', 'val']
X shape: (0, 32, 32, 3)
y shape: (0,)
